In [1]:
import os
from dotenv import load_dotenv
import dtlpy as dl
from rfdetr.util.coco_classes import COCO_CLASSES



In [2]:
use_rc_env = True
if use_rc_env:
    dl.setenv('rc')
else:
    dl.setenv('prod')
if dl.token_expired():
    dl.login()


In [4]:
if use_rc_env:
    project = dl.projects.get(project_name='HusamDimo')
else:
    project = dl.projects.get(project_name='ShadiDemo')

In [ ]:
models = project.models.list().print()


In [ ]:
# Publish your app
dpk = project.dpks.publish(local_path='model_adapter',manifest_filepath='dataloop.json')
# Install or update the application
try:
    app = project.apps.get(app_name=dpk.display_name)
    app.dpk_version = dpk.version
    app.update()
except dl.exceptions.NotFound:
    print("installing ...")
    app = project.apps.install(dpk=dpk)

In [ ]:
for model in project.models.list().all():
    print(model.name)


In [6]:
model = project.models.get(model_name = 'rf-detr')

set the lables from COCO_CLASSES

In [7]:
model.labels = list(COCO_CLASSES.values())

In [ ]:
len(COCO_CLASSES.values())


upload the weights file


In [ ]:
print(f"model status before artifact upload: {model.status}")
# Upload your weights
artifact = model.artifacts.upload(filepath='model.pth')

# Update the configuration
model.configuration['weights_filename'] = artifact.filename


model = project.models.get(model_name = 'rf-detr')
print(f"model status after artifact upload: {model.status}")

model.status = 'trained'
model.update()

# # # Deploy your model
# print("-##################\n##################\n##################\n##################\n##################\n deploy only once \n##################\n##################\n##################\n##################\n##################\n")
# model.deploy()

model = project.models.get(model_name = 'rf-detr')
print(f"model status after deploy: {model.status}")

In [ ]:
cloned_model_taxi = project.models.get(model_name='rf-detr-tex4l')
artifact = cloned_model_taxi.artifacts.upload(filepath='model.pth')

# Update the configuration
cloned_model_taxi.configuration['weights_filename'] = artifact.filename

model.update()


In [ ]:
dataset = project.datasets.get(dataset_name='nvidia-husam-clone-updated-name')
image_url = 'https://media.roboflow.com/notebooks/examples/dog-2.jpeg'
item_3 = dataset.items.upload(image_url)
prediction_3 = model.predict(item_ids=[item_3.id],dataset_id=dataset.id)
prediction_3.wait()
item_3.open_in_web()

In [24]:
model = project.models.get(model_name = 'rf-detr')
model.open_in_web()
project.datasets.get(dataset_id='67f3d54728294f8e79c43965').open_in_web()

In [21]:

# Train with data splitting and validation
train_filters = dl.Filters(field='dir', values='/train')
val_filters = dl.Filters(field='dir', values='/validation')


model = model.clone(
    model_name='zibra_pingwin_model_3',
    description='Husam trained model',
    dataset=project.datasets.get(dataset_id='67f3d54728294f8e79c43965'),
    project_id=project.id,
    train_filter=train_filters,
    validation_filter=val_filters,
)

In [ ]:
exec = model.train()

In [ ]:
s = dl.services.get(service_id="67b4cb48e26f1d1d9699a4c4")
s.log(system=True,
          follow=True)